# Parametri kamere

In [41]:
import numpy as np
import numpy.linalg as la

In [42]:
def normalize(x):
    return np.array([x[0] / x[3], x[1] / x[3], x[2] / x[3]])

modified_indices = []

def parametri_kamere(T):
    T0 = np.delete(T, 3, 1)

    if la.det(T0) < 0:
        T = -T

    T0 = np.delete(T, 3, 1)
    c1 = np.delete(T, 0, 1)
    c2 = np.delete(T, 1, 1)
    c3 = np.delete(T, 2, 1)
    T0C = np.array(T[:, 3])
    
    C = normalize(
        np.array(
            [
                la.det(c1),
                -1 * la.det(c2),
                la.det(c3),
                -1 * la.det(T0)
            ]
        )
    )
    C = np.round(C, 4)

    Q, R = la.qr(la.inv(T0))
    
    for i in range(R.shape[0]):
        if R[i, i] < 0:
            modified_indices.append(i)
            R[i, :] = R[i, :] * -1
            Q[:, i] = Q[:, i] * -1

    K = la.inv(R)
    K = K / K[2, 2]
    K = np.round(K, 4)

    A = la.inv(Q)
    A = np.round(A, 4)

    return K, A, C

#### Testiranje funkcije ParametriKamere(T) na zadatom primeru (n=11)

In [43]:
testT = np.array([
    [5, -1-2*11, 3., 18-3*11],
    [0., -1, 5., 21.],
    [0., -1., 0., 1.]
])
K, A, C = parametri_kamere(testT)

In [26]:
print("Matrica T koju smo koristili u zadatku (n=11)")
print(testT)

Matrica T koju smo koristili u zadatku (n=11)
[[  5. -23.   3. -15.]
 [  0.  -1.   5.  21.]
 [  0.  -1.   0.   1.]]


In [27]:
print("Matrica K (kalibracija)")
print(K)

Matrica K (kalibracija)
[[ 5.  3. 23.]
 [ 0.  5.  1.]
 [ 0.  0.  1.]]


In [28]:
print("Matrica A (orijentacija)")
print(A)

Matrica A (orijentacija)
[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [-0. -1.  0.]]


In [29]:
print("Matrica C (centar projektovanja)")
print(C)

Matrica C (centar projektovanja)
[10.  1. -4.]


#### Testiranje funkcije ParametriKamere(T) sa poslednjom cifrom indeksa (n=7 -> 057/2016)

In [30]:
myT = np.array([
    [5, -1-2*7, 3., 18-3*7],
    [0., -1, 5., 21.],
    [0., -1., 0., 1.]
])
K, A, C = parametri_kamere(myT)

In [31]:
print("Matrica T:")
print(myT)

Matrica T:
[[  5. -15.   3.  -3.]
 [  0.  -1.   5.  21.]
 [  0.  -1.   0.   1.]]


In [32]:
print("Matrica K (kalibracija)")
print(K)

Matrica K (kalibracija)
[[ 5.  3. 15.]
 [ 0.  5.  1.]
 [ 0.  0.  1.]]


In [33]:
print("Matrica A (orijentacija)")
print(A)

Matrica A (orijentacija)
[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [-0. -1.  0.]]


In [34]:
print("Matrica C (centar projektovanja)")
print(C)

Matrica C (centar projektovanja)
[ 6.  1. -4.]


# CameraDLP

In [40]:
def big_matrix(original, projekcija):
    mat1 = np.concatenate([
        [0, 0, 0, 0],
        -projekcija[2] * original.T,
        projekcija[1] * original.T
    ])
    mat2 = np.concatenate([
        projekcija[2] * original.T,
        [0, 0, 0, 0],
        projekcija[0] * original.T
    ])
    matrix = np.array([mat1, mat2])

    return matrix

def camera_dlp(originali, projekcije):
    C = big_matrix(originali[0], projekcije[0])

    for i, tacka in enumerate(originali):
        if i == 0:
            continue
        N = big_matrix(originali[i], projekcije[i])
        C = np.concatenate((C, N))

    _, _, V_t = la.svd(C)
    V = np.transpose(V_t)

    P = V[:, -1]
    P = P.reshape(3, 4)

    size = P.shape[0]
    for i in modified_indices:
        P[size-1-i,:] = -P[size-1-i,:]

    P = P/P[0,0]
    return P

#### Testiranje funkcije CameraDLP za primer iz zadatka (n=11):

In [36]:
testM1 = [460, 280, 250, 1]
testM2 = [50, 380, 350, 1]
testM3 = [470, 500, 100, 1]
testM4 = [380, 630, 50*11, 1]
testM5 = [30*11, 290, 0, 1]
testM6 = [580, 0, 130, 1]

M1p = [288., 251., 1.]
M2p = [79., 510., 1.]
M3p = [470., 440., 1.]
M4p = [520., 590., 1.]
M5p = [365, 388., 1.]
M6p = [365., 20., 1.]

testOriginali = np.array([testM1, testM2, testM3, testM4, testM5, testM6])
projekcije = np.array([M1p, M2p, M3p, M4p, M5p, M6p])

testRez = camera_dlp(testOriginali, projekcije)

In [37]:
print("Matrica kamere:")
print(testRez)

Matrica kamere:
[[ 1.00000000e+00 -1.09800191e+01  3.39560022e+01 -9.69203171e+03]
 [-2.67403824e+01  1.07542580e+01 -3.42971417e+01  2.03282746e+04]
 [-4.16755327e-02  7.29987634e-03 -7.21774501e-02  4.68368890e+01]]


#### Testiranje funkcije ParametriKamere(T) sa poslednjom cifrom indeksa (n=7 -> 057/2016)

In [38]:
myM1 = [460, 280, 250, 1]
myM2 = [50, 380, 350, 1]
myM3 = [470, 500, 100, 1]
myM4 = [380, 630, 50*7, 1]
myM5 = [30*7, 290, 0, 1]
myM6 = [580, 0, 130, 1]

myOriginali = np.array([myM1, myM2, myM3, myM4, myM5, myM6])
myRez = camera_dlp(myOriginali, projekcije)

In [39]:
print("Matrica kamere:")
print(myRez)

Matrica kamere:
[[ 1.00000000e+00 -2.42052067e+00  7.20398866e+00 -1.79202378e+03]
 [-2.69787692e+00  2.22875733e+00 -6.40773760e+00  2.40551107e+03]
 [-5.93886249e-03  3.86007422e-03 -1.63340689e-02  6.36125898e+00]]
